In [ ]:
# collapsing read count file by gene

import sys
from collections import Counter
import pickle as pkl
from pathlib import Path

gene_list = []
count_file = sys.argv[1]
gene_file = count_file[:-4] + '_gene.csv'

gene_dict = pkl.load(open("peptidome_gene_mapping.pkl","rb"))

g = open(count_file, 'r')
for line in g:
	elements = line.split(',')
	full_name = elements[0]
	count = elements[1]
	gene_name = gene_dict[full_name]
	for count in range(int(count)):
		gene_list.append(gene_name)
read_dict = Counter(gene_list)
sorted_reads = sorted(read_dict.items(), reverse = True, key = lambda kv:kv[1])

h = open(gene_file, 'w')
for item in sorted_reads:
	h.write(str(item[0]) + ',' + str(item[1])+'\n')

In [ ]:
# individual read counts to rpk matrix

from collections import Counter
import copy
import sys
import os
from subprocess import call

# iterative mapping function
def map(all_dict, csv_file):
	total = 0
	temp_dict = {}
	h = open(csv_file,'r')
	for line in h:
		elements = line.split(',')
		temp_dict[elements[0].strip()] = elements[1].strip()
		total += float(elements[1].strip())
	h.close()

	for item in all_dict:
		all_dict[item] += ','
		if item in temp_dict:
			all_dict[item] += str(float(temp_dict[item]) * 100000 / total)
		else:
			all_dict[item] += '0'
	return all_dict

# building initial dictionaries
library_path = 'library_fasta/'
seq_dict_pep = {}
g1 = open(library_path + 'peptidome_t7_seq.fasta', 'r')
for line in g1:
	if line[0] == '>':
		seq_dict_pep[line[1:].strip()] = ""
g1.close()
seq_dict_gene = {}
g2 = open(library_path + 'peptidome_genes.txt', 'r')
for line in g2:
	seq_dict_gene[line.strip()] = ""
g2.close()
training_path = 'all_names.csv'
training_dict = {}
peptide_name_list = []
g3 = open(training_path, 'r')
for line in g3:
	peptide_name = line.strip().split(',')[0]
	peptide_name_list.append(peptide_name)
	training_dict[peptide_name] = ""
g3.close()


# writing new files

output_file_pep = 'sample_files/' + sample_name + '_mapped.csv'
output_file_gene = 'sample_files/' + sample_name + '/' + sample_name + '_mapped_gene.csv'

# populating dictionary
gene_names = 'gene' # initialize first line
pep_names = 'peptide'

name = str(file).strip().split('/')[-1].split('.csv')[0]
gene_names += ',' + name
pep_names += ',' + name
temp_seq_dict_pep = copy.deepcopy(seq_dict_pep)
temp_seq_dict_pep = map(temp_seq_dict_pep, file)

call(['python','collapsebygene.py', file])
temp_seq_dict_gene = copy.deepcopy(seq_dict_gene)
temp_seq_dict_gene = map(temp_seq_dict_gene, file[:-4] + '_gene.csv')

# writing to output file
w1 = open(output_file_pep, 'w')
w1.write(pep_names + '\n')
for peptide in peptide_name_list:
	w1.write(str(peptide) + temp_seq_dict_pep[peptide] + '\n')
w1.close()

w2 = open(output_file_gene, 'w')
w2.write(gene_names + '\n')
for gene in temp_seq_dict_gene:
	w2.write(str(gene) + temp_seq_dict_gene[gene] + '\n')
w2.close()

In [ ]:
# taking top X features by max value

number_features = 500

rpk_file = 'all_rpk.csv'
g = open(rpk_file, 'r')
max_values = {}
header = True
for line in g:
    if header:
        header = False
        continue
    elements = line.strip().split(',')
    maximum = max([float(i) for i in elements[1:]])
    max_values[elements[0]] = maximum
max_value_list = sorted(max_values.items(), key=lambda x:x[1],reverse = True)

counter = 0
top_dict = {}
for item in max_value_list:
    if counter < number_features:
        top_dict[item[0]] = 0
    counter += 1

g = open(rpk_file, 'r')
h = open('top_features' + str(number_features) + '_rpk.csv','w')

header = True
for line in g:
    peptide = line.strip().split(',')[0]
    if header:
        h.write(line)
        header = False
        continue
    if peptide in top_dict:
        h.write(line)
g.close()
h.close()

In [ ]:
# running model iterations

number_iterations = 100

roc_input = np.linspace(0,1,201)
roc_all = []
auc_all = []

fpr_best = []
tpr_best = []
ROC_auc_best = 0
fpr_worst = []
tpr_worst = []
ROC_auc_worst = 1

# running X times
for seed_number in range(0,number_iterations):
	print("Iteration:"+str(seed_number))
	np.random.seed(seed_number)
	pos_test_list = np.random.choice(len(pos_list), len(pos_list), replace = False)
	np.random.seed(seed_number + 100)
	neg_test_list = np.random.choice(len(neg_list), len(neg_list), replace = False)
	test_predictions = []
	test_probs = []
	test_labels = []

# MODEL HERE

	fpr, tpr, thresholds = metrics.roc_curve(test_labels, test_probs, pos_label = 1.0)
	ROC_auc = metrics.auc(fpr, tpr)

	roc_all.append(np.interp(roc_input,fpr,tpr))
	auc_all.append(ROC_auc)

# save if ROC is new best / worst
	if ROC_auc > ROC_auc_best:
		fpr_best = fpr
		tpr_best = tpr
		ROC_auc_best = ROC_auc

	if ROC_auc < ROC_auc_worst:
		fpr_worst = fpr
		tpr_worst = tpr
		ROC_auc_worst = ROC_auc

roc_avg = np.average(np.array(roc_all), axis = 0)
ROC_auc_avg = np.average(auc_all)

# plotting best, worst, average
plt.rcParams["font.size"] = 11
plt.rcParams["font.family"] = "Arial"
plt.figure()
lw = 2
plt.plot(fpr_best, tpr_best, color='green', lw=lw, label='Best Iteration (AUC = %0.2f)' % ROC_auc_best)
plt.plot(roc_input, roc_avg, color='blue', lw=lw, label='Average: %i Iterations (AUC = %0.2f)' % (number_iterations,ROC_auc_avg))
plt.plot(fpr_worst, tpr_worst, color='red', lw=lw, label='Worst Iteration (AUC = %0.2f)' % ROC_auc_worst)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")